In [22]:
] activate .

LoadError: syntax: unexpected "]"

### Run all cells, the interactive plot is at the bottom

In [26]:
using Interact, StatsPlots, DataFrames, CSV, Dates;
include("plotting_functions.jl");
#WebIO.install_jupyter_nbextension() # might need to reload page afterwards in order to work

#### Choose the parameter combination to evaluate in the interactive plot (see results folder)

In [27]:
season = "summer"
date = "2021-03-16"
version = "v12"
NUM_STEPS = 24
NUM_EP = 3000
L1 = 300
L2 = 600
case = "$(season)_abort_mem-less";
#case = "$(season)_abort_mem-ini_full";

In [28]:
Input_df = CSV.read("data/$(season)_evaluation.csv", DataFrame);

In [29]:
Flow_df = CSV.read("out/$(date)_results_$(version)_$(NUM_STEPS)_$(NUM_EP)_$(L1)_$(L2)_$(case).csv", DataFrame);

In [30]:
first(Flow_df, 3)

,Temp_FH,Vol_HW,Soc_B,V_HW_plus,V_HW_minus,T_FH_plus,T_FH_minus,profits
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,20.3624,76.1425,6.84937,0.0,0.0,0.0,0.0,-0.0651839
2,20.4015,75.4713,6.84917,0.0,0.0,0.0,0.0,-0.0705452
3,20.4321,74.8,6.84896,0.0,0.0,0.0,0.0,-0.0633516


In [31]:
size(Flow_df)

(383, 22)

In [32]:
first(Input_df, 3)

,electkwh,heatingkwh,hotwaterkwh,PV_generation,Temperature,month,day,hour
,Float64,Int64,Float64,Float64,Float64,Int64,Int64,Int64
1,0.189026,0,0.36191,0.0,16.88,7,16,0
2,0.172257,0,0.0,0.0,16.775,7,16,1
3,0.159676,0,0.0,0.0,16.635,7,16,2


In [33]:
size(Input_df)

(384, 8)

In [34]:
Data_df = hcat(Flow_df[:, 1:end-2], Input_df[1:end-1, :]) # cut index + hour + last input (no action)

,Temp_FH,Vol_HW,Soc_B,V_HW_plus,V_HW_minus,T_FH_plus,T_FH_minus,profits
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,20.3624,76.1425,6.84937,0.0,0.0,0.0,0.0,-0.0651839
2,20.4015,75.4713,6.84917,0.0,0.0,0.0,0.0,-0.0705452
3,20.4321,74.8,6.84896,0.0,0.0,0.0,0.0,-0.0633516
4,20.4535,74.1288,6.84876,0.0,0.0,0.0,0.0,-0.0582118
5,20.4655,73.4576,6.84855,0.0,0.0,0.0,0.0,-0.0533789
6,20.464,72.7864,6.84835,0.0,0.0,0.0,0.0,0.0330243
7,20.4572,72.9063,6.84814,0.0,0.0,0.0,0.0,0.14369
8,20.4505,68.7586,6.84794,0.0,0.0,0.0,0.0,0.313691
9,20.4572,70.5824,6.84773,0.0,0.0,0.0,0.0,0.250575


In [35]:
size(Data_df)

(383, 28)

In [36]:
describe(Data_df)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Int64,DataType
1,Temp_FH,21.83,20.3624,21.8613,22.4002,0,Float64
2,Vol_HW,101.716,49.8342,104.019,177.382,0,Float64
3,Soc_B,8.71535,6.24368,8.73315,9.99973,0,Float64
4,V_HW_plus,0.0703962,0.0,0.0,0.400213,0,Float64
5,V_HW_minus,0.0,0.0,0.0,0.0,0,Float64
6,T_FH_plus,0.0,0.0,0.0,0.0,0,Float64
7,T_FH_minus,0.0,0.0,0.0,0.0,0,Float64
8,profits,0.0853639,-0.704775,0.0176079,0.766143,0,Float64
9,COP_FH,0.381974,0.0,0.0,5.61,0,Float64


In [37]:
@show names(Data_df);

names(Data_df) = ["Temp_FH", "Vol_HW", "Soc_B", "V_HW_plus", "V_HW_minus", "T_FH_plus", "T_FH_minus", "profits", "COP_FH", "COP_HW", "PV_DE", "B_DE", "GR_DE", "PV_B", "PV_GR", "PV_HP", "GR_HP", "B_HP", "HP_FH", "HP_HW", "electkwh", "heatingkwh", "hotwaterkwh", "PV_generation", "Temperature", "month", "day", "hour"]


In [38]:
@manipulate for start=Date(2015,7,16):Day(1):Date(2015,7,31), 
    show=1:1:8
    
    p1=bar_row(bar_PV, Data_df, start, "kWh", "Sinks of PV generation (>0) and battery load (<0)", show);
    p2=bar_row(bar_demand, Data_df, start, "kWh", "Sources of electricity demand fulfillment", show);
    p3=bar_row(bar_heat, Data_df, start, "kWh", "Sources of heat pump demand fulfillment", show);
    p4=bar_row(bar_comfort, Data_df, start, "comfort ranges", "Heat pump SOCs and comfort ranges", show);
    
    energy_use = calc_energy_use(Data_df);
    self_consumption = calc_self_consumption(Data_df);
    self_sufficiency = calc_self_sufficiency(Data_df);
    comfort_violations = calc_comfort_violations(Data_df); ;
    profit = calc_profit(Data_df);
    
        vbox(vskip(1em),
        hbox(pad(0.1em, md"*Profit [EUR]:*"), hskip(1em), profit,
            pad(1em, md"*Energy use  [kWh]:*"), hskip(2em), energy_use, 
            pad(1em, md"*Self-consumption rate [%]:*"), hskip(1em),self_consumption,
            pad(1em, md"*Self-sufficiency rate [%]:*"), hskip(1em), self_sufficiency,
            pad(1em, md"*Comfort violations [#]:*"), hskip(1em), comfort_violations),
                        
         vskip(1em),
         md"**OUTPUT:**",
         vskip(1em),
         plot(p1,p2,p3,p4, layout=(4,1), size=(show*275, 4*220)));
    end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["start"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 16, :min => 1, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{Observables.AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing), "index" => (Observable{Any} with 2 listeners. Value:
8, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "C:\\Users\\Lissy\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\Lissy\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\Lissy\\.julia\\packages\\InteractBase\\Zfu5P\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\Lissy\\.julia\\packages\\InteractBase\\Zfu5P\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\Lissy\\.julia\\packages\\Interact\\SbgIk\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x0000000081d16bd0, Task (runnable) @0x0000000081d16bd0), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"Date(\\\"2015-07-16\\\")\",\"Date(\\\"2015-07-17\\\")\",\"Date(\\\"2015-07-18\\\")\",\"Date(\\\"2015-07-19\\\")\",\"Date(\\\"2015-07-20\\\")\",\"Date(\\\"2015-07-21\\\")\",\"Date(\\\"2015-07-22\\\")\",\"Date(\\\"

### Calculate KPIs

In [51]:
gd = groupby(Data_df, ["month", "day"])
Data_df_sum = combine(gd, valuecols(gd) .=> sum, renamecols=false)

,month,day,Temp_FH,Vol_HW,Soc_B,V_HW_plus,V_HW_minus,T_FH_plus,T_FH_minus
,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,7,16,491.94,1504.71,190.12,0.0,0.0,0.0,0.0
2,7,17,511.06,2147.73,182.463,0.0,0.0,0.0,0.0
3,7,18,517.063,2194.07,197.392,0.0,0.0,0.0,0.0
4,7,19,521.66,2484.33,197.235,0.0,0.0,0.0,0.0
5,7,20,523.162,2333.21,207.055,0.0,0.0,0.0,0.0
6,7,21,523.742,2061.1,207.964,0.0,0.0,0.0,0.0
7,7,22,523.732,2701.65,220.161,0.0,0.0,0.0,0.0
8,7,23,524.145,2571.08,217.333,0.0,0.0,0.0,0.0
9,7,24,525.263,2075.31,204.833,0.0,0.0,0.0,0.0


In [60]:
@show names(Data_df_sum);

names(Data_df_sum) = ["month", "day", "Temp_FH", "Vol_HW", "Soc_B", "V_HW_plus", "V_HW_minus", "T_FH_plus", "T_FH_minus", "profits", "COP_FH", "COP_HW", "PV_DE", "B_DE", "GR_DE", "PV_B", "PV_GR", "PV_HP", "GR_HP", "B_HP", "HP_FH", "HP_HW", "electkwh", "heatingkwh", "hotwaterkwh", "PV_generation", "Temperature", "hour", "Cons_GR", "Cons_DE", "Cons_HP", "Cons_B"]


In [77]:
Data_df_sum[!,"Cons_GR"] = (Data_df_sum[!,"PV_GR"]./Data_df_sum[!,"PV_generation"])*100
Data_df_sum[!,"Cons_DE"] = (Data_df_sum[!,"PV_DE"]./Data_df_sum[!,"PV_generation"])*100
Data_df_sum[!,"Cons_HP"] = (Data_df_sum[!,"PV_HP"]./Data_df_sum[!,"PV_generation"])*100
Data_df_sum[!,"Cons_B"] = (Data_df_sum[!,"PV_B"]  ./Data_df_sum[!,"PV_generation"])*100
Data_df_sum[!, 26:end]

,PV_generation,Temperature,hour,Cons_GR,Cons_DE,Cons_HP,Cons_B
,Float64,Float64,Int64,Float64,Float64,Float64,Float64
1,41.26,497.557,276,73.6089,11.7935,1.9528,7.64479
2,57.962,616.541,276,78.9665,7.7514,1.79984,6.48222
3,31.333,605.127,276,66.2223,15.6375,4.73938,8.40082
4,65.23,584.663,276,81.6419,7.793,1.45161,4.11352
5,54.98,549.67,276,80.6118,7.94238,2.45913,3.98669
6,70.075,520.598,276,83.3229,6.38576,2.29432,2.99699
7,49.967,532.807,276,82.2846,8.45359,1.84725,2.41451
8,67.556,557.646,276,83.7186,6.5634,2.46777,2.25019
9,58.93,557.993,276,80.0099,8.73825,2.24142,4.01039


In [ ]:
Data_df_sum_cons_pivot_GR = Data_df_sum[["Cons_GR", "month","day"]].pivot(index='day', columns='month', values= 'Cons_GR');
Data_df_sum_cons_pivot_DE = Data_df_sum[["Cons_DE", "month","day"]].pivot(index='day', columns='month', values= 'Cons_DE');
Data_df_sum_cons_pivot_HP = Data_df_sum[["Cons_HP", "month","day"]].pivot(index='day', columns='month', values= 'Cons_HP');
Data_df_sum_cons_pivot_B = Data_df_sum[["Cons_B", "month","day"]].pivot(index='day', columns='month', values= 'Cons_B');

100-Data_df_sum_cons_pivot_GR.max(axis=0)

In [85]:
unstack(Data_df_sum[!,["Cons_B", "month","day"]], "day", 
    "month", "Cons_B")

,day,7
,Int64,Float64?
1,16,7.64479
2,17,6.48222
3,18,8.40082
4,19,4.11352
5,20,3.98669
6,21,2.99699
7,22,2.41451
8,23,2.25019
9,24,4.01039
